# Hierarchical reinforcement learning (HRL)
- 강화학습은 많은 상태, 액션 공간을 잘 scaling하지 못함 
- 차원의 저주에 빠지기 쉬움 

### 많은 상태공간, 액션공간을 다루기 위해 큰 문제들을 작은 sub문제들로 계층적으로 분해 
#### - 예시 
- 목표: 학교에서 집에 가기 
    - sub문제
        - 교문 나서기
        - 택시 잡기 

### HRL에서 사용되는 여러 방법 
#### - 상태 공간 분해 (state-space decomposition)
- 상태공간을 여러 subspace로 분해 
- 더 작아진 subspace에서 문제를 해결 
- 빠르게 탐험 가능 
    - 전체 상태 공간을 탐험하지 않아도 됨 
    
#### - 상태 추상화 (state-abstraction)
- 에이전트는 일부 변수를 무시 
    - 현재 상태공간에서 현재 subtask를 달성하는데 관련없는 변수 무시 
    
#### - 시간적 추상화 (temporal-abstraction)
- 액션 sequence와 액션 set들이 그룹화됨 
    - 하나의 step을 여러 step으로 나눔 

## * MAXQ Value Function Decomposition
#### : HRL에서 많이 사용되는 알고리즘 
### 가치 함수를 각 subtask들에 해당하는 가치 함수 set으로 분해 
#### - 택시 환경 예시 
<img src="./image/taxi.PNG">

- 목적: 4개의 위치(R,G,Y,B) 중 한 곳에서 승객을 태우고(pick) 다른 위치에 내려주는 것(drop off)
- 보상 
    - 올바른 곳에 내려주면 +20 
    - 올바르지 못한(illegal) pick, drop off -10 
    - 매 time step마다 -1 
    
#### - subtask 분해 
<img src="./image/maxq_graph.png" width=500>   
 
 
- 모든 subtask들이 계층적으로 정렬됨 
- node: subtask 또는 기본 액션 
- edge: subtask가 호출할 수 있는 child subtask


- Navigate(t)
    - 현재 위치에서 다른 위치로 이동 
    - 원초(primitive) 액션  4개 
        - 북, 남, 동, 서 
        
        
- Get
    - 현재 위치에서 승객을 태울 위치로 이동 
    - 원초 액션 1개 
        - Pickup 
    - subtask 1개 
        - Navigate(t)
   
   
- Put 
    - 현재 위치에서 승객을 내려줄 위치로 이동
    - 원초 액션 1개 
        - Putdown 
    - subtask 1개 
        - Navigate(t)
    
    
- root 
    - 전체 task 
    
#### - 수식화 
- MDP M은 subtask들의 set으로 분해 
    - ($M_0$, $M_1$, $M_2$, ... $M_n$)
    - $M_0$: root task 
    - $M_1$... : subtasks
    
    
- subtask $M_i$는 semi MDP
    - 상태 $S_i$
    - 액션 $A_i$
    - 확률 전이 함수 
        - $P^x_y (s', N | s, a)$
    - 기대 보상 함수 
        - $\bar{R}(s, a) = V^\pi(a, s)$
            - 이 때, $V^\pi(a, s)$: 상태 s에서 child task $M_a$의 투사된(projected) 가치함수 
            
    - 만약, 액션 a가 원초 액션이라면, 
        - $V^\pi(a, s)$: 상태s에서 수행한 액션 a의 기대 즉시 보상 
        - $V^\pi(a, s)= \sum_{s'}P(s'|s,a). R(s'|s,a)$
        
    - 벨만 방정식에 따라 다시 쓰면, 
        $$V^\pi(i, s) = V^\pi(\pi_i(s), s) + \sum_{s',N} P^\pi_i(s', N|s, \pi_i(s)) \gamma^N V^\pi(i, s')$$
        
    - 상태-액션 가치 함수 Q 
        $$Q^\pi(i, s, a) = V^\pi(a, s) + \sum_{s',N}P^\pi_i(s', N | s, a)\gamma^NQ^\pi(i, s', \pi(s'))$$
        
    - completion function: subtask $M_i$를 완료하는 기대할인축적보상 
         $$C^\pi(i, s, a) = \sum_{s', N}P^\pi_i(s', N | s, a)\gamma^NQ^\pi(i, s', \pi(s'))$$
         
    - 이에따른, Q함수
         $$Q^\pi(i, s, a) = V^\pi(a, s) + C^\pi(i, s, a)$$
         
    - 재정의된 가치 함수 
    $$V^\pi(i, s) = \begin{cases}Q^\pi(i, s, \pi(s)) \text{  if i is composite}\\
                                   \sum_{s'}P(s' | s, i)R(s', | s, i) \text{  if i is primitive}\end{cases}$$
            - 가치함수와 완료함수의 합 

# Inverse reinforcement learning
- 기존 강화학습은 주어진 보상함수를 통해 최적의 정책을 찾는 것 

### 최적의 정책은 주어지고 보상함수를 찾는 것 
#### - 이유 
- 보상함수를 구성하는것은 복잡 
- 잘못된 보상함수는 에이전트가 나쁜행동을 하게 만듦

#### - 항상 적절한 보상함수를 알 수는 없지만 옳은 정책은 안다 
- 옳은 정책: 각 상태에서의 올바른 행동 
    - 인간 전문가를 통해 에이전트에 입력 
    - 에이전트는 보상함수를 학습하려 함 
    
### 예시 
#### - 걷는 것을 배울 때 
- 무한한 행동에 대한 보상함수를 설계하는 것은 어려움 
- 대신 전문가가 demonstration(시범, 설명)을 제공하여 에이전트가 보상함수를 학습하도록 함